In [41]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import astropy.units as u
from astroduet.duet_sensitivity import calc_exposure
import numpy as np
from matplotlib import pyplot as plt
from astroduet.bbmag import bb_abmag_fluence, bb_abmag

import astroduet.config as config
from astroduet.background import background_pixel_rate


from astropy.visualization import quantity_support
import matplotlib
font = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 22}

matplotlib.rc('font', **font)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [42]:
duet = config.Telescope()
duet.config_list

['baseline',
 'classic',
 'minimum_mass',
 'fine_plate',
 'large_aperture',
 'reduced_baseline']

In [43]:
duet = config.Telescope(config='classic')


In [45]:
psd_model = duet.psf_model()

In [22]:
# 5-sigma limiting magnitude in 1, 2, 3, and 4 stacked frames.
# Assume read noise is 3 e-

exposure = 300*u.s
siglimit=5
dmag = 0.1

for tel_type in duet.config_list:
    print('------')
    print(tel_type)
    print()
    duet = config.Telescope(config=tel_type)
    duet.read_noise=3

    [bgd_band1, bgd_band2] = background_pixel_rate(duet, low_zodi=True)
    print('Sky background rate per pixel band1: {}'.format(bgd_band1))
    print('Sky background rate per pixel band2: {}'.format(bgd_band2))
    print()
    bandone = duet.bandpass1
    bandtwo = duet.bandpass2
    for nframes in np.arange(1, 5):
        snr = 100
        swiftmag = 20 
        while snr > siglimit:
            swiftmag += dmag
            band1_fluence, foo = bb_abmag_fluence(duet =duet, swiftmag=swiftmag*u.ABmag, bbtemp=15e3*u.K)
            band1_rate = duet.fluence_to_rate(band1_fluence)
            snr = duet.calc_snr(exposure, band1_rate, bgd_band1, nint=nframes)
        bbmag1, foo = bb_abmag(swiftmag=swiftmag*u.ABmag, bbtemp=15e3*u.K, bandone = bandone)
        print('Band 1 {} {}-σ magnitude limit: {}'.format(nframes*exposure, siglimit, bbmag1))

    for nframes in np.arange(1, 5):
        snr = 100
        swiftmag = 20 
        while snr > siglimit:
            swiftmag += dmag
            foo, band2_fluence = bb_abmag_fluence(duet =duet, swiftmag=swiftmag*u.ABmag, bbtemp=15e3*u.K)
            band2_rate = duet.fluence_to_rate(band2_fluence)
            snr = duet.calc_snr(exposure, band2_rate, bgd_band2, nint=nframes)
        foo, bbmag2 = bb_abmag(swiftmag=swiftmag*u.ABmag, bbtemp=15e3*u.K, bandtwo = bandtwo)
        print('Band 2 {} {}-σ magnitude limit: {}'.format(nframes*exposure, siglimit, bbmag2))

    # Stacked channels
    # 5-sigma limiting magnitude in 1, 2, 3, and 4 stacked frames.

    # Account for the fact that you're co-adding the two frames here:
    duet.read_noise = (2**0.5) * 3
    tot_bgd_rate = bgd_band1 + bgd_band2
    for nframes in np.arange(1, 5):
        snr = 100
        swiftmag = 20 
        while snr > siglimit:
            swiftmag += dmag
            band1_fluence, band2_fluence = bb_abmag_fluence(duet =duet, swiftmag=swiftmag*u.ABmag, bbtemp=15e3*u.K)
            band1_rate = duet.fluence_to_rate(band1_fluence)
            band2_rate = duet.fluence_to_rate(band2_fluence)
            src_rate = band1_rate + band2_rate

            snr = duet.calc_snr(exposure, src_rate, tot_bgd_rate, nint=nframes)
        bbmag1, bbmag2 = bb_abmag(swiftmag=swiftmag*u.ABmag, bbtemp=15e3*u.K, bandone = bandone, bandtwo=bandtwo)
        print('Stacked Bands {} {}-σ magnitude limit: {}'.format(nframes*exposure, siglimit, bbmag1))
    print()




------
baseline

Sky background rate per pixel band1: 0.030369732491096913 ph / s
Sky background rate per pixel band2: 0.2478588509265617 ph / s

Band 1 300.0 s 5-σ magnitude limit: 22.70047009109242 mag(AB)
Band 1 600.0 s 5-σ magnitude limit: 23.200470091092427 mag(AB)
Band 1 900.0 s 5-σ magnitude limit: 23.40047009109243 mag(AB)
Band 1 1200.0 s 5-σ magnitude limit: 23.600470091092433 mag(AB)
Band 2 300.0 s 5-σ magnitude limit: 21.888257363507968 mag(AB)
Band 2 600.0 s 5-σ magnitude limit: 22.288257363507974 mag(AB)
Band 2 900.0 s 5-σ magnitude limit: 22.488257363507977 mag(AB)
Band 2 1200.0 s 5-σ magnitude limit: 22.68825736350798 mag(AB)
Stacked Bands 300.0 s 5-σ magnitude limit: 22.80047009109242 mag(AB)
Stacked Bands 600.0 s 5-σ magnitude limit: 23.200470091092427 mag(AB)
Stacked Bands 900.0 s 5-σ magnitude limit: 23.50047009109243 mag(AB)
Stacked Bands 1200.0 s 5-σ magnitude limit: 23.600470091092433 mag(AB)

------
classic

Sky background rate per pixel band1: 0.0284860875465870

In [27]:
# FOM: Time to 22 ABmag for this configuration for DUET1
# Old FOM for this 


siglimit=10
swiftmag = 21
dmag = 0.05
bbmag1 = 5*u.ABmag
bbmag_target = 22*u.ABmag
while bbmag1 < bbmag_target:
    swiftmag+=dmag
    bbmag1, bbmag2 = bb_abmag(swiftmag=swiftmag*u.ABmag, bbtemp=15e3*u.K, bandone = bandone, bandtwo=bandtwo)
print(f"""-----
        Swift AB mag {swiftmag}
        DUET1 AB mag {bbmag1}
        DUET2 AB mag {bbmag2}""")



for ind, tel_type in enumerate(duet.config_list):
    print()
    duet = config.Telescope(config=tel_type)
    duet.read_noise=3
    duet.info()

    siglimit=10
    bandone = duet.bandpass1
    bandtwo = duet.bandpass2

    # Low Zodi, 10-sigma case
    [bgd_band1, bgd_band2] = background_pixel_rate(duet, low_zodi=True)
    band1_fluence, foo = bb_abmag_fluence(duet = duet, swiftmag=swiftmag*u.ABmag, bbtemp=15e3*u.K)
    band1_rate = duet.fluence_to_rate(band1_fluence)
    req_exp = calc_exposure(siglimit, band1_rate, bgd_band1, duet.read_noise, duet.neff)
    print('Low-Zodi {}-sig, {} seconds'.format(siglimit, req_exp.value))
    
    [bgd_band1, bgd_band2] = background_pixel_rate(duet, med_zodi=True)
    band1_fluence, foo = bb_abmag_fluence(duet = duet, swiftmag=swiftmag*u.ABmag, bbtemp=15e3*u.K)
    band1_rate = duet.fluence_to_rate(band1_fluence)
    req_exp = calc_exposure(siglimit, band1_rate, bgd_band1, duet.read_noise, duet.neff)
    print('Med-Zodi {}-sig, {} seconds'.format(siglimit, req_exp.value))



    # High Zodi, 10-sigma case.
    [bgd_band1, bgd_band2] = background_pixel_rate(duet, high_zodi=True)
    band1_fluence, foo = bb_abmag_fluence(duet = duet, swiftmag=swiftmag*u.ABmag, bbtemp=15e3*u.K)
    band1_rate = duet.fluence_to_rate(band1_fluence)
    req_exp = calc_exposure(siglimit, band1_rate, bgd_band1, duet.read_noise, duet.neff)
    print('High-Zodi {}-sig, {} seconds'.format(siglimit, req_exp.value))

    # High Zodi, 5-sigma case
    siglimit=5
    [bgd_band1, bgd_band2] = background_pixel_rate(duet, high_zodi=True)
    band1_fluence, foo = bb_abmag_fluence(duet = duet, swiftmag=swiftmag*u.ABmag, bbtemp=15e3*u.K)
    band1_rate = duet.fluence_to_rate(band1_fluence)

    req_exp = calc_exposure(siglimit, band1_rate, bgd_band1, duet.read_noise, duet.neff)
    print('High-Zodi {}-sig, {} seconds'.format(siglimit, req_exp.value))



-----
        Swift AB mag 22.000000000000014
        DUET1 AB mag 22.000470091092396 mag(AB)
        DUET2 AB mag 21.58825736350795 mag(AB)

-----
        DUET Telescope State: baseline
        Physical Entrance Pupil: 26.0 cm
        Effective EPD: 24.2 cm
        Effective Area: 459.9605804120816 cm2
        Transmission Efficiency: 0.8166518036622619
        
        Pixel size: 6.4 arcsec
        PSF FWHM: 5.0 arcsec
        Pointing jitter: 5.0 arcsec
        Effective PSF FWHM: 7.0710678118654755 arcsec
        N_eff: 3.3581908798935647

        Band 1: {'eff_wave': <Quantity 202.56878682 nm>, 'eff_width': <Quantity 53.32814342 nm>}
        Bandpass 1: [175.90471511 229.23285853] nm
        Band 2: {'eff_wave': <Quantity 281.7531854 nm>, 'eff_width': <Quantity 68.16239088 nm>}
        Bandpass 2: [247.67198996 315.83438085] nm

        Dark current: 0.0115 ph / s
        Read noise (RMS per read): 3
        -----
        
Low-Zodi 10-sig, 472.7113937270214 seconds
Med-Zodi 10-si